# Objective: Identify the first day when the tracker stall happens

Objective 1: Find the SPC's with Tracker Stall from all plants. <br>
Objective 2: Find the first date of Tracker Stall of each SPC... -> No dates should be defined manually. <br>
Objective 3: From the first date of stall consider 2 weeks prior data and find the "Damper Issues" and "Slew Gear Issues".

In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import datetime
import os, fnmatch
from pvlib import solarposition

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = {'SiteName':['Roswell','whitney'],'latitude':[33.44,36.36], 'Longitude':[-104.444,-120.129]}
data2 = pd.DataFrame(data, columns = ['SiteName','latitude','Longitude'])

In [3]:
%%time
loc = r'C:\Users\ghyhvara\Box Sync\Work_2019\Projects\Project 1\DataSet\RootCauseAnalysisData'

import os, pandas as pd
os.chdir(loc)
file_list = list()

for file in os.listdir():
    if file.endswith('.csv'):
        df = pd.read_csv(file)
        df['filename'] = file
        file_list.append(df)

all_files = pd.concat(file_list, axis=0, ignore_index=True)
all_files['Time'] = pd.to_datetime(all_files['Time'],format="%Y-%m-%d %H:%M:%S")

Wall time: 7.46 s


In [4]:
%%time
all_files['year_month'] = all_files['Time'].dt.month.astype(str)+ '.' + all_files['Time'].dt.year.astype(str)
all_files['Deviation_Degree'] = abs(np.subtract(all_files['position'],all_files['target']))

Wall time: 3.22 s


1. Normally motor current is between the 2 - 3 Amps, if it is going beyond 4 Amps then it need to be monitored. <br>
2. When the deviation (Target Angle - Actual Angle) > 5 it should be monitored. <br>

Based on the above information if the below code satifies both condition then we can expect there will be a Tracker Stall.

In [5]:
%%time
features = all_files.columns
if ('maxMotorCurrent' in features) & ('motorCurrent' in features):
        mean1=all_files['maxMotorCurrent'].mean()
        mean2=all_files['motorCurrent'].mean()
        if np.isnan(mean1):
            all_files['MC']=all_files['motorCurrent']
        elif np.isnan(mean2):
            all_files['MC']=all_files['maxMotorCurrent']
        else:
            if mean1>mean2:
                all_files['MC']=all_files['maxMotorCurrent']
            if mean1<mean2:
                all_files['MC']=all_files['motorCurrent']
        all_files.drop(['maxMotorCurrent','motorCurrent'], axis=1,inplace=True)

elif ('maxMotorCurrent' in features) & ('motorCurrent' not in features):
    all_files['MC']=all_files['maxMotorCurrent']
    all_files.drop(['maxMotorCurrent'], axis=1,inplace=True)

elif ('maxMotorCurrent' not in features) & ('motorCurrent' in features):
    all_files['MC']=all_files['motorCurrent']
    all_files.drop(['motorCurrent'], axis=1,inplace=True)

else:
    print('No motor current data for analysis')

Wall time: 334 ms


In [6]:
%%time
all_files.loc[(all_files['MC'].ge(4) & all_files['Deviation_Degree'].ge(5)),'motor_status'] = 'Stall'
all_files['motor_status'] = all_files['motor_status'].fillna('No_Stall')

Wall time: 446 ms


In [14]:
Analysis6.groupby(['filename','year_month'])['motor_status'].value_counts().to_frame('count')

count
filename                                           year_month motor_status       
Roswell_10B_SPCNo_0134_SPCSrNo_SPCSG20155301641... 1.2018     Stall          2869
                                                   2.2018     Stall          7422
                                                   3.2018     Stall          8363
                                                   4.2018     Stall          7224
                                                   5.2018     Stall          8285
                                                   6.2018     Stall          7715
                                                   7.2018     Stall          7093
Roswell_2B_SPCNo_0221_SPCSrNo_SPCSG20160303618_... 4.2018     Stall          4407
                                                   5.2018     Stall          8149
                                                   6.2018     Stall          7542
                                                   7.2018     Stall          7298
Roswell_2B_SPCNo_0240_SPCSrNo_SPCSG20160303698.csv 1.2018     Stall           611
                                                   11.2017    Stall          1325
                                                   2.2018     Stall             1
                                                   5.2018     Stall          1969
                                                   6.2018     Stall          6084
                                                   7.2018     Stall          7230
Roswell_5A_SPCNo_0213_SPCSrNo_SPCSG20160304313_... 5.2018     Stall          6278
                                                   6.2018     Stall          7423
                                                   7.2018     Stall          7225
Roswell_6B_SPCNo_0201_SPCSrNo_SPCSG20160302102_... 1.2018     Stall           102
                                                   2.2018     Stall          6137
                                                   3.2018     Stall          7551
                                                   4.2018     Stall          7898
                                                   5.2018     Stall          8179
                                                   6.2018     Stall          7743
                                                   7.2018     Stall          6951
whitney_point_11_SPCNo_0107_SPCSrNo_SPCSG201614... 3.2019     Stall          6133
                                                   4.2019     Stall          8070
                                                   5.2019     Stall          7506
...                                                                           ...
whitney_point_3_SPCNo_0227_SPCSrNo_SPCSG2016060... 6.2019     Stall          5163
                                                   9.2018     Stall            88
whitney_point_4_SPCNo_0213_SPCSrNo_SPCSG2016130... 2.2019     Stall           361
                                                   3.2019     Stall          7695
                                                   4.2019     Stall          8057
                                                   5.2019     Stall          7056
                                                   6.2019     Stall          5179
whitney_point_5_SPCNo_0243_SPCSrNo_SPCSG2016130... 1.2019     Stall          6521
                                                   2.2019     Stall          7399
                                                   3.2019     Stall          7822
                                                   4.2019     Stall          7853
                                                   5.2019     Stall          6542
whitney_point_6_SPCNo_0102_SPCSrNo_SPCSG2016050... 2.2019     Stall          3862
                                                   3.2019     Stall          7822
                                                   4.2019     Stall          7998
                                                   5.2019     Stall          7315
                                                   6.2019 

In [13]:
%%time
sample_data = all_files[all_files['motor_status']=='Stall']


Analysis6 = sample_data
df = Analysis6.groupby(['filename','year_month'])['motor_status'].value_counts().to_frame('count').reset_index()
df = df.sort_values(['year_month'], ascending = False, axis=0)
df = df[df['count']>2000]
df_test = df.groupby(['filename'])['year_month','count'].last().reset_index()
final = sample_data.merge(df_test, on=['filename','year_month'], how = 'inner')



df_test = df_test.reset_index()
print(df_test)
# print(df)

    index                                           filename year_month  count
0       0  Roswell_10B_SPCNo_0134_SPCSrNo_SPCSG2015530164...     1.2018   2869
1       1  Roswell_2B_SPCNo_0221_SPCSrNo_SPCSG20160303618...     4.2018   4407
2       2  Roswell_2B_SPCNo_0240_SPCSrNo_SPCSG20160303698...     6.2018   6084
3       3  Roswell_5A_SPCNo_0213_SPCSrNo_SPCSG20160304313...     5.2018   6278
4       4  Roswell_6B_SPCNo_0201_SPCSrNo_SPCSG20160302102...     2.2018   6137
5       5  whitney_point_11_SPCNo_0107_SPCSrNo_SPCSG20161...     3.2019   6133
6       6  whitney_point_11_SPCNo_0133_SPCSrNo_SPCSG20155...     3.2019   7096
7       7  whitney_point_11_SPCNo_0232_SPCSrNo_SPCSG20155...     1.2019   6736
8       8  whitney_point_3_SPCNo_0227_SPCSrNo_SPCSG201606...     1.2019   6832
9       9  whitney_point_4_SPCNo_0213_SPCSrNo_SPCSG201613...     3.2019   7695
10     10  whitney_point_5_SPCNo_0243_SPCSrNo_SPCSG201613...     1.2019   6521
11     11  whitney_point_6_SPCNo_0102_SPCSrNo_SPCSG2

In [8]:
harsha

NameError: name 'harsha' is not defined

In [15]:
%%time

RecentStall = final.groupby(['filename'])['Time','MC'].first()
all_files = all_files.sort_values(['filename','Time'])
all_files['StallTime'] = all_files['filename'].map(RecentStall['Time'])
#Filtering the data for the last 2 weeks.
FilteredData = all_files.loc[(all_files['Time']>(all_files['StallTime'].subtract(pd.to_timedelta('15d')))) & (all_files['Time']<all_files['StallTime'])]


Wall time: 1.65 s


In [ ]:
RecentStall

In [16]:
%%time
# This will be called from our database, currently we have created the dictionary but in realtime this code will be useful to
# connect with database
data2['SiteName'] = data2['SiteName'].str.lower()
data2.set_index('SiteName',inplace=True)
FilteredData['SiteName'] = FilteredData.filename.str.lower().apply(lambda x: data2.index[pd.Series(map(x.startswith,data2.index.tolist()))][0])
FilteredData = FilteredData.merge(data2,left_on='SiteName',right_index=True,how='outer')

Wall time: 8.83 s


In [17]:
%%time

East_stall = []
West_stall = []
Tracker_stall = []
# fulltable = 
names  = FilteredData.filename.unique()
for name in names:
    x = FilteredData[FilteredData.filename==name]
#     x.Time = pd.to_datetime(x.Time)
    x.Time = x.Time.dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
    dt_start = x.Time.min()
    dt_end = x.Time.max()

    high_motor_current_indicating_already_stalled = 4.5
    high_motor_current_indicating_damper_fault = 4
    high_motor_current_cutoff = 3.5
    slope_motor_current_cutoff = 3.0
    slope_threshold_to_detect_failure = 0.016  # this roughly corresponds to 0.5 amp increase in 15 days (0.5/15=0.03)
    no_of_days_used_for_slope_calc = abs((dt_end - dt_start).days)
    num_days_to_avg = 5
    motor_current_ts = x['MC']
    SiteTimeUTC = x.index
    lat = x['latitude']
    lon = x['Longitude']

    sunpos_df = solarposition.get_solarposition(x.Time, latitude=lat, longitude=-lon).reset_index()
    sunpos_df = sunpos_df[['Time','apparent_elevation','elevation']]
    x = x.merge(sunpos_df,on='Time',how='outer')

# Calculate the mean motor currents when the tracker angles are close to the extension limits (for eg. 60E and 60W)
# This criteria is used to detect damper failure issues which show peaks in motor currents at these tracker angles
    trackerTargetAngles = x['target'].values
    trackerActualAngles = x['position'].values

    maxTrackerAngle = max(trackerTargetAngles)
    IndicesOfExtremeTargetPositions_E = np.where((trackerTargetAngles < -(maxTrackerAngle - 3)))
    IndicesOfExtremeTargetPositions_W = np.where((trackerTargetAngles > (maxTrackerAngle - 3)) )
    avg_motorCurrentAtExtremeTargetPositions_E = float( np.mean(motor_current_ts.values[IndicesOfExtremeTargetPositions_E]))
    avg_motorCurrentAtExtremeTargetPositions_W = float( np.mean(motor_current_ts.values[IndicesOfExtremeTargetPositions_W]))
#     print(avg_motorCurrentAtExtremeTargetPositions_E, avg_motorCurrentAtExtremeTargetPositions_W)



    x = x[x['apparent_elevation']>=0]
    x['Day']= x['Time'].dt.month.astype(str)+ '.' + x['Time'].dt.day.astype(str)+ '.' + x['Time'].dt.year.astype(str)
    motor_current_avg_day = x.groupby(['filename','Day'])['MC'].mean().reset_index()
    x = x.merge(motor_current_avg_day,on=['filename','Day'],how='outer')

#     motor_current_avg_day_list = x.MC_y.fillna(method = 'ffill')
#     print(motor_current_avg_day_list.isnull().sum())
#     print(x.head())

    motor_current_min_day = x.MC_x.min()
    motor_current_min_day_list = round(float(motor_current_min_day),3)
    
    motor_current_avg_day_2weeks = np.mean(x.MC_x)
    motor_current_min_day_avg_over_2weeks = np.mean(motor_current_min_day_list)
#     print(name,'|',avg_motorCurrentAtExtremeTargetPositions_E,'|',high_motor_current_indicating_damper_fault,'|',motor_current_min_day_avg_over_2weeks)
#     print('*'*30)
#     print(name,'|',avg_motorCurrentAtExtremeTargetPositions_W,'|',high_motor_current_indicating_damper_fault,'|',motor_current_min_day_avg_over_2weeks)
    
 #-------------------------------------------------------------------------------------------       
    
#     y =  motor_current_avg_day_2weeks
#     x = range(1, len(str(y)) + 1)
#     slope, intercept = np.polyfit(x, y, 1)
    
#----------------------------------------------------------------------------------------------
## Damper Slipage 
    if avg_motorCurrentAtExtremeTargetPositions_W > high_motor_current_indicating_damper_fault and (abs(avg_motorCurrentAtExtremeTargetPositions_W-motor_current_min_day_avg_over_2weeks) > 1):
#         print(['West Stall',name, avg_motorCurrentAtExtremeTargetPositions_W]) ## use this to demonstrate the tracker stall value in west side
        stall_W = ['West Stall',name, avg_motorCurrentAtExtremeTargetPositions_W]
        West_stall.append(stall_W)

    
    if avg_motorCurrentAtExtremeTargetPositions_E > high_motor_current_indicating_damper_fault and (abs(avg_motorCurrentAtExtremeTargetPositions_E-motor_current_min_day_avg_over_2weeks) > 1):
#         print(['East Stall',name, avg_motorCurrentAtExtremeTargetPositions_E])
        stall_E = ['East Stall',name, avg_motorCurrentAtExtremeTargetPositions_E]
        East_stall.append(stall_E)


        if stall_E[1] == stall_W[1]:
            stall_Tracker = ['Tracker Stall -- ',stall_W[1], motor_current_avg_day_2weeks]
#             print('Tracker Stall -- ',stall_E[1])
#             stall_Tracker = ['Tracker Stall -- ',stall_E[1]]
            Tracker_stall.append(stall_Tracker)

    
#     if motor_current_avg_day_2weeks >= high_motor_current_indicating_already_stalled:
#         print(name,'-------','stalled')

Wall time: 3.15 s


In [25]:
avg_motorCurrentAtExtremeTargetPositions_E

3.013170212765957

In [18]:
# pd.set_option('display.max_colwidth',-1)

# #Tracker_stall
# Tracker_stall = pd.DataFrame(Tracker_stall, columns=['Stall', 'Filename','MC'])
# #East_Stall
# East_stall = pd.DataFrame(East_stall, columns=['Stall_Side','Filename','MC'])
# East_stall = East_stall[East_stall.Filename.ne(Tracker_stall.Filename)]
# East_stall = East_stall[East_stall.ne(Tracker_stall)]
# #West_Stall
# West_stall = pd.DataFrame(West_stall, columns=['Stall_Side', 'Filename','MC'])
# West_stall = West_stall[West_stall.Filename.ne(Tracker_stall.Filename)]

# print(Tracker_stall)
# print('*'*80)
# print(East_stall)
# print('*'*80)
# print(West_stall)



In [19]:
pd.set_option('display.max_colwidth',-1)
#East_Stall
East_stall = pd.DataFrame(East_stall, columns=['Stall_Side','Filename','MC'])
# East_stall = East_stall[East_stall.ne(Tracker_stall)]
#West_Stall
West_stall = pd.DataFrame(West_stall, columns=['Stall_Side', 'Filename','MC'])
#Tracker_stall
Tracker_stall = pd.DataFrame(Tracker_stall, columns=['Stall', 'Filename','MC'])

print(East_stall)
print('*'*80)
print(West_stall)
print('*'*80)
print(Tracker_stall)

   Stall_Side                                                 Filename  \
0  East Stall  Roswell_2B_SPCNo_0240_SPCSrNo_SPCSG20160303698.csv        
1  East Stall  whitney_point_3_SPCNo_0227_SPCSrNo_SPCSG20160602210.csv   
2  East Stall  whitney_point_4_SPCNo_0213_SPCSrNo_SPCSG20161305201.csv   

         MC  
0  4.094890  
1  4.936287  
2  4.859265  
********************************************************************************
   Stall_Side                                                  Filename  \
0  West Stall  Roswell_2B_SPCNo_0240_SPCSrNo_SPCSG20160303698.csv         
1  West Stall  whitney_point_11_SPCNo_0133_SPCSrNo_SPCSG20155000042.csv   
2  West Stall  whitney_point_3_SPCNo_0227_SPCSrNo_SPCSG20160602210.csv    
3  West Stall  whitney_point_5_SPCNo_0243_SPCSrNo_SPCSG20161305504.csv    
4  West Stall  whitney_point_6_SPCNo_0102_SPCSrNo_SPCSG20160500084.csv    
5  West Stall  whitney_point_6_SPCNo_0141_SPCSrNo_SPCSG20161400541.csv    

         MC  
0  4.729845  
1  4.421167 

From the data signatures I observed tracker information is been captured for every <b>5mins </b>, 

For 1 hour => (60mins / 5mins) = 12 Times data is captured per hour <br>
For 1 Day => (12times x 24hours) = 288 Times data is captured in one day.<br>
For 1 Week => (288times x 7days) = 2016 Times data is captured in one week.<br>

2000 Times: (2000/60 = 33hours, 33hours*5 = 166hours, 166hours/24 = 7 days)

<strike>if the timeperiod between the tracker stalls is more than 7 days then next tracker stall is considered </strike>
<br>
If the latest tracker stall time is more than 7 days then it is considered.

# Questions

(Situation already Happened)
1. This 'RootCauseAnalysis" is done for those trackers which were already stalled and we are trying to find the latest stall date and based on two week signatures we are finiding the issue. -- No Predictions were made here <br>
2. When the tracker was already stalled why are we getting the data signatures?



# Idea

(What is about to Happen)
1. Build a model which will read prior two weeks information from current date and help you predict the breakdown of the Tracker, including the reason for breakdown.

In [ ]:
import pandas as pd
data = pd.read_csv(r"C:\Users\ghyhvara\Box Sync\Work_2019\Projects\Project 1\Algorithm\whitney_point_NCU_2_SPC_SPCSG20154802722_DAMPER_60E.csv")

In [ ]:
data = data.rename(columns={'position':'trackerActualAngles', 'target':'trackerTargetAngles'})

In [ ]:
trackerTargetAngles = data['trackerTargetAngles']
trackerActualAngles = data['trackerActualAngles']



In [ ]:
noOfOccuranceWithAngleDiffGreaterThan5deg = sum(
    1 if abs(trackerTargetAngles[i] - trackerActualAngles[i]) > 5 else 0 for i in
    range(0, len(trackerTargetAngles)))

In [ ]:
noOfOccuranceWithAngleDiffGreaterThan5deg

In [ ]:
len(trackerTargetAngles)

In [ ]:
import numpy as np
sum(np.diff(trackerTargetAngles)==0)